In [311]:
from ds4400_final_project.dataset.constants import DATASET_FOLDER
from pathlib import Path
from typing import Tuple, Dict
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [297]:
def load_data_from_file(csv_filename: str) -> Tuple[np.array, np.array, Dict[int, str], Dict[str, int]]:
	""" Load the CSV file from the dataset folder. """
	file = str(Path(DATASET_FOLDER) / csv_filename)
	features_list = np.genfromtxt(file, dtype=None, encoding=None, delimiter=",", skip_header=1, usecols=range(2, 60))
	features = np.array([list(x) for x in features_list])

	# Create a mapping between a numeric value and genre
	index_genre_map = {i: genre for i, genre in enumerate(np.unique(features[:,-1]))}
	genre_index_map = {value: key for key, value in index_genre_map.items()}

	# split the inputs and their labels
	x = normalize(features[:,:57])
	# y = normalize(np.array([genre_index_map[genre] for genre in features[:,-1]]).reshape(-1, 1))

	# take labels and make vector where the value at the index is 1 and everything else is 0
	def make_one_hot(idx):
		v = np.zeros((10,))
		v[idx] = 1.0
		return v
	# y = np.array([make_one_hot(genre_index_map[genre]) for genre in features[:,-1]])
	y = np.array([genre_index_map[genre] for genre in features[:,-1]])

	return x, y, index_genre_map, genre_index_map

In [298]:
# import the data from the 3 seconds and 30 seconds features CSV
X_3, y_3, index_genre_map_3, genre_index_map_3 = load_data_from_file("features_3_sec.csv")
X_30, y_30, index_genre_map_30, genre_index_map_30 = load_data_from_file("features_30_sec.csv")

In [299]:
# split all the data into training and testing sets
TEST_SIZE = 0.33
RANDOM_STATE = 42

x_3_train, x_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=TEST_SIZE, random_state=RANDOM_STATE)
x_30_train, x_30_test, y_30_train, y_30_test = train_test_split(X_30, y_30, test_size=TEST_SIZE, random_state=RANDOM_STATE)
x_3_train, x_3_test, x_30_train, x_30_test = map(torch.FloatTensor, [x_3_train, x_3_test, x_30_train, x_30_test])
y_3_train, y_3_test, y_30_train, y_30_test = map(torch.LongTensor, [y_3_train, y_3_test, y_30_train, y_30_test])

In [300]:
class ShallowModel(nn.Module):

    def __init__(self, num_hidden: int):
        super().__init__()
        self.hidden = nn.Linear(57, num_hidden)
        self.output = nn.Linear(num_hidden, 10)
        self.softmax = nn.LogSoftmax(dim=1)
        

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return torch.softmax(x, dim=1)
        # return self.softmax(x)

In [301]:
class DeepModelTwoHiddenLayer(nn.Module):

    def __init__(self, num_hidden_nodes_1: int, num_hidden_nodes_2: int):
        super().__init__()
        self.hidden1 = nn.Linear(57, num_hidden_nodes_1)
        self.hidden2 = nn.Linear(num_hidden_nodes_1, num_hidden_nodes_2)
        self.output = nn.Linear(num_hidden_nodes_2, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = self.output(x)
        return x
        #return self.softmax(x)

In [302]:
class GTZANDataset(Dataset):

    def __init__(self, x_data, y_data):
        self._x_data = x_data
        self._y_data = y_data

    def __len__(self):
        return len(self._x_data)
        
    def __getitem__(self, idx):
        return [self._x_data[idx], self._y_data[idx]]

In [317]:
from torchvision import datasets

def train_model(model, loss_func, x_train, y_train, epochs: int, lr: float, l2_reg: float, batch_size: int, verbose = False):
	dataset = GTZANDataset(x_train, y_train)
	dataloader = DataLoader(dataset, batch_size=batch_size)
	size = len(training_data)

	optimizer = torch.optim.SGD(params=model.parameters(), lr=lr, weight_decay=l2_reg)

	# set the model to training mode
	model.train()

	# optimize the model
	for epoch in range(epochs):

		print(f"\nEpoch {epoch+1}\n-------------------------------") if verbose else None

		for batch, (x, y) in enumerate(dataloader):

			y_pred = model(x)
			# print(y_pred.size())
			# print(y.size())

			loss = loss_function(y_pred, y)

			# backpropogation
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			if verbose and batch % 100 == 0:
				# print(f"Batch {batch}: train loss: {loss.item()}")
				loss, current = loss.item(), batch * len(x)
				print(f"Batch [{batch:03}]: loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



	# set the model to evaluation mode
	model.eval()
	return model

In [315]:
EPOCHS = 10
LEARNING_RATE = 0.001
L2_REGULARIZATION = 0.001
BATCH_SIZE = 10

model_3 = train_model(
    model=ShallowModel(num_hidden=100),
    x_train=x_3_train,
    y_train=y_3_train,
    epochs=EPOCHS,
    lr=LEARNING_RATE,
    l2_reg=L2_REGULARIZATION,
    batch_size=BATCH_SIZE,
    verbose=True,
)

TypeError: train_model() missing 1 required positional argument: 'loss_func'

In [318]:
EPOCHS = 5
LEARNING_RATE = 0.001
L2_REGULARIZATION = 0.001
BATCH_SIZE = 200

model_deep_one_layer_3 = train_model(
    model=DeepModelTwoHiddenLayer(num_hidden_nodes_1=256, num_hidden_nodes_2=128),
    loss_func=nn.NLLLoss(),
    x_train=x_3_train,
    y_train=y_3_train,
    epochs=EPOCHS,
    lr=LEARNING_RATE,
    l2_reg=L2_REGULARIZATION,
    batch_size=BATCH_SIZE,
    verbose=True,
)

NameError: name 'training_data' is not defined

In [ ]:
model.eval()
y_pred = model(x_3_test)
after_train = loss_function(y_pred.squeeze(), y_3_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 2.3030574321746826
